In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
data2001 = pd.read_csv('./pems08/d08_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

,Timestamp,Station,District,Freeway #,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,Avg Occupancy,Avg Speed
0,02/28/2018 00:00:00,801230,8,10,W,ML,0.845,40,100,101.0,0.0212,69.4
1,02/28/2018 00:00:00,801231,8,10,W,OR,NaN,20,100,10.0,NaN,NaN
2,02/28/2018 00:00:00,801232,8,10,E,ML,0.996,39,100,147.0,0.0929,69.5
3,02/28/2018 00:00:00,801233,8,10,E,OR,NaN,20,100,7.0,NaN,NaN
4,02/28/2018 00:00:00,801234,8,10,W,ML,0.658,40,100,84.0,0.0269,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...
609691,02/28/2018 23:55:00,827521,8,215,N,OR,NaN,9,0,NaN,NaN,NaN
609692,02/28/2018 23:55:00,827529,8,210,W,FR,NaN,6,50,13.0,NaN,NaN
609693,02/28/2018 23:55:00,827530,8,210,W,OR,NaN,3,100,3.0,NaN,NaN
609694,02/28/2018 23:55:00,827537,8,210,E,FR,NaN,6,100,17.0,NaN,NaN


In [4]:
comm = np.load('pems08_comm.npy')
comm

array([801230, 801231, 801232, 801233, 801234, 801236, 801237, 801238,
       801239, 801241, 801242, 801243, 801244, 801245, 801248, 801249,
       801250, 801252, 801255, 801259, 801260, 801261, 801262, 801265,
       801266, 801267, 801268, 801269, 801270, 801271, 801273, 801275,
       801277, 801279, 801281, 801282, 801283, 801284, 801285, 801286,
       801287, 801288, 801320, 801321, 801323, 801324, 801326, 801327,
       801329, 801331, 801335, 801337, 801339, 801340, 801342, 801343,
       801345, 801346, 801348, 801349, 801351, 801352, 801354, 801355,
       801357, 801358, 801360, 801361, 801363, 801364, 801366, 801367,
       801369, 801370, 801372, 801373, 801375, 801376, 801378, 801379,
       801381, 801383, 801385, 801387, 801389, 801390, 801392, 801418,
       801421, 801422, 801425, 801428, 801429, 801431, 801434, 801435,
       801437, 801445, 801448, 801449, 801452, 801456, 801469, 801472,
       801473, 801476, 801481, 801484, 801485, 801487, 801488, 801489,
      

In [5]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    try:
        # Read the CSV file, specifying the usecols parameter to only load the columns of interest
        data = pd.read_csv(
            file_path, 
            header=None, 
            usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
            names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
            compression='gzip'
        )

        # Convert the Timestamp column to datetime format and set it as the index
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data.set_index('Timestamp', inplace=True)

        # Pivot the table to get Stations as columns and Total Flow as cell values
        pivot_data = data.pivot(columns='Station', values='Avg Flow')

        return pivot_data

    except EOFError:
        print(f"Error processing file: {file_path}")
        return None

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2001, 3, 1)  # 开始日期
# end_date = date(2012, 10, 2)   # 结束日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems08/d08_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
#     if year==2012 and month==10 and day==1:
#         continue
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems08_all_common_flow.csv')

In [ ]:
combined_data

In [1]:
import pandas as pd
pems08 = pd.read_csv('pems08_all_common_flow.csv')

# Convert the 'date' column to datetime
pems08['date'] = pd.to_datetime(pems08['date'])

# Set the 'date' column as the index
pems08.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems08_hourly = pems08.resample('H').sum()

# Reset the index if you want the 'date' column back
pems08_hourly.reset_index(inplace=True)
pems08_hourly.to_csv('pems08_h.csv',index=False)


# Convert the 'date' column to datetime
pems08 = pd.read_csv('pems08_all_common_flow.csv')
pems08['date'] = pd.to_datetime(pems08['date'])

# Set the 'date' column as the index
pems08.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems08_daily = pems08.resample('D').sum()

# Reset the index if you want the 'date' column back
pems08_daily.reset_index(inplace=True)
pems08_daily.to_csv('pems08_d.csv',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25608\3086877500.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pems08_hourly = pems08.resample('H').sum()
